# Le data mining vs le machine learning

- Data Mining : recherche d’information et de patterns cachés dans une base de données isolée ou en croisant plusieurs bases. C’est l’être humain qui explore et prend des décisions.
- Le data mining s’appuie sur des techniques et algorithmes issus de la statistique et de la gestion de bases de données.
- Les données exploitées ont souvent une taille considérable.


# Quelques exemples

- Prestation de service : identification des paramètres de prédisposition à la résiliation (churn)
- Marketing: identification de segments distincts de consommateurs 
- Production : identification de paramètres induisant des défaillances
- Pharmaceutique: classification de médicaments selon le profil chimique


# Le machine learning

- Machine Learning : Techniques de calcul permettant aux ordinateurs d’apprendre à exécuter des tâches en s’inspirant de patterns perçus sur des données, avec peu d’intervention humaine. Les tâches se résument souvent à de la prédiction.

![image](./images/ml1.png)
    

![image](./images/trend-ml.png)

# Les outils

![image](./images/poll-ml.png)

# Les types de machine learning

- Les algorithmes de machine learning peuvent être rassemblés dans 3 groupes principaux :
   - L'apprentissage supervisé
   - Cible (variable dépendante qualitative ou quantitative)
   - Prédicteurs (variables indépendantes)
   - Régression, arbre de decision, foret aléatoire, kNN, regression logistique, SVM…

- L'apprentissage non supervisé
    - Pas de variable dépendante, pas de modélisation
    - Classer des individus dans différents groups ou trouver des associations
    - Apriori algorithm, K-means.

- L'apprentissage par renforcement
    - Apprentissage pour prendre des décisions spécifique afin d’aboutir à un but précis
    - Markov Decision Process

# L’analyse de données et le machine learning avec Python

Le package la plus adaptée pour faire du data mining et du machine learning est la package **scikit-learn**


Ce package propose des fonctions prédéfinies pour un grand nombre de méthodes

- La classification : SVM, plus proches voisins, random forest…
- Les régressions : linéaire, ridge, Lasso…
- Le clustering : k-means…
- L’analyse de données :  ACP, DA…

Dans le cadre de cette formation, l'objectf n'est pas de décrire la théorie des méthodes mais plutôt de comprendre l'utilisation de Python pour les appliquer.

# L’apprentissage supervisé avec Scikit-Learn
Les méthodes d’apprentissage supervisé sont les méthodes actuellement les plus
utilisées en data science. Il s’agit d’essayer de prédire une variable cible et d’utiliser
différentes méthodes pour arriver à cette fin.
Nous allons illustrer ces méthodes de traitement de données avec du code et des
cas pratiques.

### Les données et leur transformation
Ce jeu de données est décrit en détail au début du chapitre 4. Pour rappel, il est composé de 3333 individus et de 18 variables. Il est stocké dans un fichier csv, nommé telecom.csv, accessible dans le répertoire Data. On le récupère en utilisant Pandas :

In [1]:
import pandas as pd
import numpy as np

In [2]:
churn=pd.read_csv("./data/telecom.csv")

## Les données

Ce jeu de données n’a pas de données manquantes et nous allons devoir effectuer
quelques transformations pour l’adapter à nos traitements. Nous voyons par exemple qu’il est composé de trois colonnes object.

Nous pouvons afficher les statistiques descriptives pour les colonnes object :

In [3]:
churn.describe(include="object").transpose()

,count,unique,top,freq
State,3333,51,WV,106
Phone,3333,3333,385-1214,1
Int'l Plan,3333,2,no,3010
VMail Plan,3333,2,no,2411
Churn?,3333,2,False.,2850


## Transformation des données

On voit que les données sont toutes binaires. 

Pour les variables binaires, il nous suffit de les recoder avec Scikit-Learn pour obtenir des données exploitables. Par
ailleurs, il existe une autre variable qualitative dans notre jeu de données, Area Code,
qui est numérique mais avec trois modalités :

In [10]:
churn["Area Code"].value_counts()

415    1655
510     840
408     838
Name: Area Code, dtype: int64

## La préparation des données

Nous allons utiliser le processus de traitement classique pour transformer nos
données avec Scikit-Learn. Dans ce cas, nous n’avons pas de données manquantes,
nous travaillons donc sur la transformation des variables qualitatives.

In [35]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,Imputer, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer 

## Nouveauté : ColumnTransformer

On peut maintenant tranformer plusieurs colonnes simultanément

Essayez d'utliser cet outil :

In [53]:
# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = [ 'VMail Message','Day Mins', 'Day Calls', 'Day Charge',
       'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
       'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
       'CustServ Calls']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_features = ['Area Code', "Int'l Plan",
       'VMail Plan']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore',categories='auto'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)],
sparse_threshold=0.1)

In [54]:
x = preprocessor.fit_transform(churn)

In [55]:
x.shape

(3333, 21)

## Prédire l’attrition des clients
Lorsqu’on veut prédire une variable binaire, on devra avoir une colonne du type
binaire. On préfère généralement un codage 0/1 afin de garder un type entier simple à gérer. 

Les variables explicatives x auront été préparées de manière intelligente afin de bien appliquer nos modèles.

On crée donc x et y :

In [57]:
y = churn["Churn?"] 

## Séparation des données

Pour la séparation, on utilise la fonction train_test_split() de Scikit-Learn.

Cette fonction permet de créer automatiquement autant de structures que nécessaire
à partir de nos données. 

Elle utilise une randomisation des individus et ensuite une séparation en fonction d’un paramètre du type test_size :

In [58]:
# on importe la fonction
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3, stratify = y)

Dans certains cas, il peut arriver qu’il y ait une forte disparité de distribution des
modalités entre les proportions d’acceptation et de refus. On peut vouloir faire en
sorte que les répartitions des modalités de y soient égales dans les différents échantillons,
on pourra alors utiliser une stratification. On va utiliser une stratification en
prenant y comme base pour effectuer la stratification :

Ainsi les deux échantillons _train et _test ont la même distribution

In [59]:
y_test.value_counts(normalize=True)

False.    0.855
True.     0.145
Name: Churn?, dtype: float64

In [60]:
y_train.value_counts(normalize=True)

False.    0.855122
True.     0.144878
Name: Churn?, dtype: float64

## Le choix et l’ajustement de l’algorithme

Tout au long de ce Notebook, nous allons essayer d'ajouter un nouveau modèle, il s'agit du modèle GBM
```python
from sklearn.ensemble import GradientBoostingClassifier
```

In [61]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
# ajouter le 3ème modèle

Ensuite, on crée un objet à partir de la classe du modèle en lui fournissant les
hyperparamètres dont il a besoin :

In [62]:
modele_rf=RandomForestClassifier(n_estimators=100)
modele_knn=KNeighborsClassifier(n_neighbors=5)
# ajouter le 3ème modèle

Dans ce cas, on prend les hyperparamètres par défaut.

On peut ensuite ajuster notre modèle en utilisant les données :

In [63]:
modele_rf.fit(x_train,y_train)
modele_knn.fit(x_train,y_train)
# ajouter le 3ème modèle
pass

Une fois qu’on a estimé les paramètres du modèle, on va pouvoir extraire des
informations. De nouveaux attributs de chaque classe apparaissent, ils se terminent par le symbole underscore _ :

In [64]:
modele_rf.feature_importances_

array([0.02412936, 0.14868924, 0.03070257, 0.13137712, 0.06658133,
       0.03035028, 0.06729782, 0.03726518, 0.03147922, 0.0425611 ,
       0.04253456, 0.05824639, 0.04639394, 0.11272828, 0.0049688 ,
       0.00549283, 0.00460445, 0.04184707, 0.04218884, 0.01444341,
       0.01611822])

Ce qui va nous intéresse avant tout, c’est de prédire avec notre modèle. Pour cela nous allons utiliser la méthode .predict() :

In [65]:
y_predict_rf = modele_rf.predict(x_test)
y_predict_knn = modele_knn.predict(x_test)
# ajouter le 3ème modèle

On obtient ainsi une valeur prédite pour les éléments de notre échantillon de
validation.

## Les indicateurs pour valider un modèle
La partie validation d’un modèle d’apprentissage supervisé est extrêmement
importante. L’objectif d’un modèle d’apprentissage supervisé est de prédire une
valeur la plus proche possible de la réalité. Nous différencions trois types d’indices
en fonction du type de variable cible. Tous les indicateurs de qualité du modèle sont
stockés dans le module *metrics* de Scikit-Learn.

## Le pourcentage de bien classés
Il s’agit de l’indicateur le plus connu. On le nomme accuracy. Il est calculé à partir du rapport entre le nombre d’individus bien classés et le nombre total d’individus dans l’échantillon.

In [67]:
from sklearn.metrics import accuracy_score, recall_score

accuracy_modele_rf = accuracy_score(y_test,y_predict_rf)
accuracy_modele_knn = accuracy_score(y_test,y_predict_knn)
print("Pourcentage de bien classés pour le modèle RF : %.3f" %(accuracy_modele_rf))
print("Pourcentage de bien classés pour le modèle kNN :%.3f" %(accuracy_modele_knn))

Pourcentage de bien classés pour le modèle RF : 0.951
Pourcentage de bien classés pour le modèle kNN :0.889


## La matrice de confusion
Il s’agit d’un autre indicateur important pour juger de la qualité d’un modèle, il n’est pas défini par une seule valeur mais par une matrice dans laquelle on peut lire le croisement entre les valeurs observées et les valeurs prédites à partir du modèle. 

Pour calculer cette matrice, on pourra utiliser :

In [68]:
from sklearn.metrics import confusion_matrix
confusion_matrix_rf=confusion_matrix(y_test,y_predict_rf)
confusion_matrix_knn=confusion_matrix(y_test,y_predict_knn)
print("Matrice de confusion pour le modèle RF :",
confusion_matrix_rf, sep="\n")
print("Matrice de confusion pour le modèle kNN :",
confusion_matrix_knn, sep="\n")
# ajouter le 3ème modèle

Matrice de confusion pour le modèle RF :
[[847   8]
 [ 41 104]]
Matrice de confusion pour le modèle kNN :
[[852   3]
 [108  37]]


## Le rappel (recall), la précision et le f1-score

Scikit-Learn possède des fonctions pour chacun de ces indicateurs, mais il peut
être intéressant d’utiliser une autre fonction qui les affiche pour chaque classe :

tp / (tp + fn)

In [69]:
from sklearn.metrics import classification_report
print("Rapport pour le modèle RF :",
      classification_report(y_test,y_predict_rf) ,sep="\n")

Rapport pour le modèle RF :
              precision    recall  f1-score   support

      False.       0.95      0.99      0.97       855
       True.       0.93      0.72      0.81       145

    accuracy                           0.95      1000
   macro avg       0.94      0.85      0.89      1000
weighted avg       0.95      0.95      0.95      1000



In [70]:
print("Rapport pour le modèle kNN :",
      classification_report(y_test,y_predict_knn) ,sep="\n")

Rapport pour le modèle kNN :
              precision    recall  f1-score   support

      False.       0.89      1.00      0.94       855
       True.       0.93      0.26      0.40       145

    accuracy                           0.89      1000
   macro avg       0.91      0.63      0.67      1000
weighted avg       0.89      0.89      0.86      1000



## L’aire sous la courbe ROC
La courbe ROC est un indicateur important mais on préfère souvent une valeur plutôt
qu’une courbe afin de comparer nos modèles. Pour cela, on utilise l’aire sous la courbe
ROC (AUC). Cette aire est calculée directement à partir de la courbe ROC. Ainsi, un
modèle aléatoire aura une AUC de 0.5 et un modèle parfait aura une AUC de 1.

In [72]:
from sklearn.metrics import roc_auc_score
auc_modele_rf=roc_auc_score(y_test, modele_rf.predict_proba(x_test)[:,1])
auc_modele_knn=roc_auc_score(y_test,modele_knn.predict_proba(x_test)[:,1])

print("Aire sous la courbe ROC pour le modèle RF :" ,auc_modele_rf)
print("Aire sous la courbe ROC pour le modèle kNN :" ,auc_modele_knn)
# ajouter le 3ème modèle

Aire sous la courbe ROC pour le modèle RF : 0.8856503327283727
Aire sous la courbe ROC pour le modèle kNN : 0.7829723734623917


## La validation croisée
Jusqu’ici nous avons utilisé des indicateurs basés sur une seule occurrence de test. Ceci veut dire qu’on ne teste notre modèle que sur un seul échantillon.

Une approche alternative souvent utilisée est la validation croisée. Celle-ci est en fait basée sur la répétition de l’estimation et de la validation sur des données différentes.

Pour obtenir ce cv-score, on utilise :

In [73]:
from sklearn.model_selection import cross_val_score

scores_rf = cross_val_score(modele_rf, x_train, y_train, cv=5, scoring='roc_auc')
scores_knn = cross_val_score(modele_knn, x_train, y_train, cv=5, scoring='roc_auc')

print("AUC pour RF : %.3f (+/- %.3f)"% (scores_rf.mean(), scores_rf.std() * 2))
print("AUC pour kNN : %.3f (+/- %.3f)"% (scores_knn.mean(),scores_knn.std() * 2))
# ajouter le 3ème modèle

AUC pour RF : 0.925 (+/- 0.042)
AUC pour kNN : 0.788 (+/- 0.036)


## L’ajustement des hyperparamètres d’un modèle

L’une des tâches du data scientist est de trouver le meilleur modèle possible. La
plupart des modèles de machine learning ont des hyperparamètres. Il s’agit de paramètres
du modèle qui sont définis en amont de l’ajustement.

Scikit-Learn propose une classe GridSearchCV permettant d’implémenter cette
recherche d’hyperparamètres :

In [74]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

On va donc devoir définir les hyperparamètres que l’on souhaite tester. Pour cela,
on utilisera un dictionnaire d’hyperparamètres, par exemple :

In [75]:
dico_param= {"max_depth":[3,5,7,10], "n_estimators":[10,20,50,100]}

On va encore utiliser l’accuracy pour valider notre modèle. Finalement, nous allons
utiliser une validation croisée à cinq groupes pour valider les résultats.
Le nouvel objet est le suivant :

In [76]:
recherche_hyper = GridSearchCV(RandomForestClassifier(), 
                               dico_param, 
                               scoring="roc_auc",cv=5)

Une fois qu’on a créé cet objet, on peut lui joindre les données afin d’estimer les
meilleurs paramètres du modèle.

Cette étape peut être très longue.

In [77]:
recherche_hyper.fit(x_train, y_train)
pass

In [78]:
print(recherche_hyper.best_params_)

{'max_depth': 7, 'n_estimators': 50}


In [79]:
print(recherche_hyper.best_score_)

0.9249526284658357


In [80]:
pd.DataFrame(recherche_hyper.cv_results_)[["params","mean_test_score"]].sort_values("mean_test_score",
                                                                                   ascending= False)

,params,mean_test_score
10,"{'max_depth': 7, 'n_estimators': 50}",0.924953
13,"{'max_depth': 10, 'n_estimators': 20}",0.924517
15,"{'max_depth': 10, 'n_estimators': 100}",0.923651
14,"{'max_depth': 10, 'n_estimators': 50}",0.923573
11,"{'max_depth': 7, 'n_estimators': 100}",0.922636
9,"{'max_depth': 7, 'n_estimators': 20}",0.917848
5,"{'max_depth': 5, 'n_estimators': 20}",0.916776
6,"{'max_depth': 5, 'n_estimators': 50}",0.916051
12,"{'max_depth': 10, 'n_estimators': 10}",0.915637
7,"{'max_depth': 5, 'n_estimators': 100}",0.914839


## La construction d’un pipeline de traitement

Bien souvent vous allez être amené à enchaîner des traitements sur des données.
On peut bien sûr développer son code de manière à suivre les étapes une à une mais il est souvent plus intéressant de créer des suites de traitements automatisées avec Scikit-Learn. 

Ces suites de traitements sont appelées pipeline. Ils simplifieront votre code et permettront de passer en production simplement.

Ainsi, on va pouvoir faire une analyse en composantes principales suivies d’un
algorithme de plus proches voisins directement dans un pipeline :

In [81]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

In [82]:
pipe=Pipeline(steps=[("acp",PCA(n_components=8)),("knn",KNeighborsClassifier())])
pipe2=Pipeline(steps=[("acp",PCA(n_components=8)),("rf",RandomForestClassifier())])

pipe.fit(x_train, y_train)
pipe2.fit(x_train, y_train)
pass

C:\Users\s4d-asus-14\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


On a ainsi enchaîné deux traitements. Si on cherche des sorties liées à chacune
des étapes, on pourra le faire simplement. Par exemple, si l’objectif est d’extraire la part de variances expliquées par les composantes de l’analyse en composantes principales, on fera :

In [83]:
pipe.named_steps["acp"].explained_variance_ratio_

array([0.13683744, 0.13247386, 0.12931759, 0.12657744, 0.09241154,
       0.06856807, 0.06799951, 0.06631507])

In [84]:
pipe.predict(x_test)

array(['False.', 'False.', 'False.', 'False.', 'False.', 'False.',
       'False.', 'False.', 'False.', 'False.', 'False.', 'False.',
       'False.', 'False.', 'False.', 'False.', 'False.', 'False.',
       'False.', 'False.', 'False.', 'False.', 'True.', 'False.',
       'False.', 'False.', 'False.', 'False.', 'False.', 'True.',
       'False.', 'False.', 'False.', 'False.', 'False.', 'False.',
       'False.', 'False.', 'False.', 'False.', 'False.', 'False.',
       'False.', 'False.', 'True.', 'False.', 'False.', 'False.',
       'False.', 'False.', 'False.', 'False.', 'False.', 'False.',
       'False.', 'False.', 'False.', 'False.', 'False.', 'True.',
       'False.', 'False.', 'False.', 'False.', 'False.', 'False.',
       'False.', 'False.', 'True.', 'False.', 'False.', 'False.',
       'False.', 'False.', 'False.', 'False.', 'False.', 'False.',
       'False.', 'False.', 'False.', 'False.', 'False.', 'False.',
       'False.', 'True.', 'False.', 'False.', 'False.', 'False.',
 

## Trouver la meilleure combinaison d’hyperparamètres dans un pipeline

Essayons de trouver la meilleure combinaison d’hyperparamètres dans un pipeline.
Dans le cadre de cet exemple, nous utiliserons les SVM (support vector machines,
également appelés séparateurs à vaste marge ou machines à vecteurs de support).
Ce sont des méthodes assez complexes dans leur principe mais simples dans leur
mise en oeuvre.

Les hyperparamètres d’un modèle de SVM sont assez nombreux. Les plus importants
étant le noyau choisi (linéaire, polynomial, sigmoïd, RBF…), les paramètres de
ces noyaux (le degré pour le cas polynomiale, gamma…) et le C pour la marge floue.

Dans notre exemple, on utilisera la fonction make_pipeline qui est équivalente
à la classe précédente avec quelques simplifications :

In [85]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

# construction du pipeline basé sur deux approches
mon_pipe=make_pipeline(PCA(), SVC(probability=True))

# construction du dictionnaire des paramètres
# (attention utilisation de __)
param_grid = dict(pca__n_components=[5, 10, x_train.shape[1]],
                  svc__C= [1, 10, 100, 1000],
                  svc__kernel= ['sigmoid', 'rbf'],
                  svc__gamma= [0.001, 0.0001])

# on construit l’objet GridSearch et on estime les hyper-paramètres
# par validation croisée
grid_search_mon_pipe = GridSearchCV(mon_pipe, param_grid = param_grid, scoring = "roc_auc", cv = 4)

In [86]:
grid_search_mon_pipe.fit(x_train,y_train)
pass

C:\Users\s4d-asus-14\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [87]:
# la meilleure combinaisons de paramètres est :
grid_search_mon_pipe.best_params_

{'pca__n_components': 21,
 'svc__C': 1000,
 'svc__gamma': 0.001,
 'svc__kernel': 'rbf'}

In [88]:
grid_search_mon_pipe.best_score_

0.8929986172369004

Les meilleurs hyperparamètres obtenus en utilisant l’aire sous la courbe ROC sont
la combinaison C = 10, gamma = 0.0001, un noyau RBF pour les SVM et dix composantes
pour notre analyse en composantes principales.

Dans ce code, on définit les hyperparamètres associés à une méthode du pipeline
avec un double underscore : __.

L’utilisation des pipelines de Scikit-Learn va devenir rapidement une étape cruciale
de vos développements en Python.

## Passer en production votre modèle d’apprentissage supervisé

### Persistance de modèle avec Scikit-Learn

Python possède plusieurs outils pour la persistance d’objets, c’est-à-dire pour stocker
des objets dans des fichiers. Les objets de Scikit-Learn sont aussi dans cette
situation. On utilise un format pickle qui aura l’extension .pkl.

Par exemple, si nous voulons sauvegarder le dernier pipeline de traitement, nous
allons utiliser :

In [ ]:
from sklearn.externals import joblib
joblib.dump(grid_search_mon_pipe, './data/modele_grid_pipe.pkl')

Une fois ce modèle stocké, on peut très bien le réutiliser dans un autre cadre. Si
nous créons un nouveau notebook, nous allons utiliser :



In [ ]:
from sklearn.externals import joblib
grid_search_mon_pipe = joblib.load('../data/modele_grid_pipe.pkl')

On peut ensuite appliquer le modèle avec tous les paramètres qui ont été appris :


```python
grid_search_mon_pipe.predict(x_test)
```

L’utilisation d’un fichier Pickle dans un notebook est une technique assez simple et courante.